In [7]:
import requests
import json
import pandas as pd
import time

def get_address_ByIp(ip, url = "https://qifu.baidu.com/ip/geo/v1/district"):
        print(ip)
        param = {'ip':ip,
                'json':'true',
                
                } 
        ree = requests.get(url, params = param)
        try:
                re = json.loads(ree.text.replace("\\"," "))
                re = re.get("data")
        except Exception as e:
                print(f"json解析出错,网站返回结果为:{ree.text}")
                return None, None
        print(re)
        if not isinstance(re, dict):
                return None, None
        time.sleep(0.5)
        return (re.get("prov"), re.get("city"))


def get_address_ByIp2(ip, url= 'http://whois.pconline.com.cn/ipJson.jsp'):
        print(ip)
        param = {'ip':ip,
                'json':'true',
                
                } 
        ree = requests.get(url, params = param)
        try:
                re = json.loads(ree.text.replace("\\"," "))
        except Exception as e:
                print(f"json解析出错,网站返回结果为:{ree.text}")
                return None, None
        print(re)
        if not isinstance(re, dict):
                return None, None
        time.sleep(0.5)
        return (re.get("pro"), re.get("city"))

In [8]:
ipDataDf = pd.read_csv("./data.csv", sep='\t')
ipDataDf.head()

,fc,pk,ak,state,client_ip,active_time
0,tn1czr,15k4eg6e,c0cbf11e3e9b,ACTIVATED,115.194.10.49,2024-02-20T02:24:57.000Z
1,tn1czr,15k4eg6e,c0cbf11e3e9e,ACTIVATED,112.17.238.73,2024-03-14T04:22:27.000Z
2,tn1czr,15k4eg6e,c0cbf11e3ec9,ACTIVATED,183.136.152.218,2024-02-23T03:04:13.000Z
3,tn1czr,15k4eg6e,c0cbf11e3f4b,ACTIVATED,223.104.161.212,2024-03-05T08:44:46.000Z
4,tn1czr,15k4eg6e,c0cbf11e3e9d,ACTIVATED,61.153.149.52,2024-01-08T03:36:51.000Z


In [35]:
ipDataDf["pro"], ipDataDf["city"]= zip(*ipDataDf["client_ip"].apply(get_address_ByIp))
ipDataDf.head()

115.194.10.49
{'continent': '亚洲', 'country': '中国', 'zipcode': '310008', 'timezone': 'UTC+8', 'accuracy': '区县', 'owner': '中国电信', 'isp': '中国电信', 'source': '数据挖掘', 'areacode': 'CN', 'adcode': '330102', 'asnumber': '4134', 'lat': '30.229060', 'lng': '120.168903', 'radius': '4.9616', 'prov': '浙江省', 'city': '杭州市', 'district': '上城区'}
112.17.238.73
{'continent': '亚洲', 'country': '中国', 'zipcode': '310000', 'timezone': 'UTC+8', 'accuracy': '城市', 'owner': '中国移动', 'isp': '中国移动', 'source': '数据挖掘', 'areacode': 'CN', 'adcode': '330100', 'asnumber': '56041', 'lat': '30.255611', 'lng': '120.208335', 'radius': '129.2092', 'prov': '浙江省', 'city': '杭州市', 'district': ''}
183.136.152.218
{'continent': '亚洲', 'country': '中国', 'zipcode': '315100', 'timezone': 'UTC+8', 'accuracy': '区县', 'owner': '中国电信', 'isp': '中国电信', 'source': '数据挖掘', 'areacode': 'CN', 'adcode': '330212', 'asnumber': '4134', 'lat': '29.781966', 'lng': '121.527107', 'radius': '22.7884', 'prov': '浙江省', 'city': '宁波市', 'district': '鄞州区'}
223.104.16

,fc,pk,ak,state,client_ip,active_time,pro,city
0,tn1czr,15k4eg6e,c0cbf11e3e9b,ACTIVATED,115.194.10.49,2024-02-20T02:24:57.000Z,浙江省,杭州市
1,tn1czr,15k4eg6e,c0cbf11e3e9e,ACTIVATED,112.17.238.73,2024-03-14T04:22:27.000Z,浙江省,杭州市
2,tn1czr,15k4eg6e,c0cbf11e3ec9,ACTIVATED,183.136.152.218,2024-02-23T03:04:13.000Z,浙江省,宁波市
3,tn1czr,15k4eg6e,c0cbf11e3f4b,ACTIVATED,223.104.161.212,2024-03-05T08:44:46.000Z,浙江省,宁波市
4,tn1czr,15k4eg6e,c0cbf11e3e9d,ACTIVATED,61.153.149.52,2024-01-08T03:36:51.000Z,浙江省,宁波市


In [48]:
# ipDataDf.loc[(ipDataDf["city"] == "") | (ipDataDf["pro"] == ""), ["pro", "city"]] = zip(ipDataDf.loc[(ipDataDf["city"] == "") | (ipDataDf["pro"] == ""), "client_ip"].apply(get_address_ByIp2))
for client_ip in ipDataDf.loc[(ipDataDf["city"] == "") | (ipDataDf["pro"] == ""), "client_ip"]:
    pro, city = get_address_ByIp2(client_ip)
    if pro is not None and pro != "":
        ipDataDf.loc[ipDataDf["client_ip"]==client_ip, "pro"] = pro
    if city is not None and city != "":
        ipDataDf.loc[ipDataDf["client_ip"]==client_ip, "city"] = city
ipDataDf.loc[(ipDataDf["city"] == "") | (ipDataDf["pro"] == ""), :]

124.160.153.126
{'ip': '124.160.153.126', 'pro': '浙江省', 'proCode': '330000', 'city': '杭州市', 'cityCode': '330100', 'region': '', 'regionCode': '0', 'addr': '浙江省杭州市 联通', 'regionNames': '', 'err': ''}
36.113.114.98
{'ip': '36.113.114.98', 'pro': '江苏省', 'proCode': '320000', 'city': '南京市', 'cityCode': '320100', 'region': '', 'regionCode': '0', 'addr': '江苏省南京市 电信', 'regionNames': '', 'err': ''}
123.246.178.71
{'ip': '123.246.178.71', 'pro': '辽宁省', 'proCode': '210000', 'city': '辽阳市', 'cityCode': '211000', 'region': '', 'regionCode': '0', 'addr': '辽宁省辽阳市 电信', 'regionNames': '', 'err': ''}
1.180.4.7
{'ip': '1.180.4.7', 'pro': '内蒙古', 'proCode': '150000', 'city': '呼和浩特市', 'cityCode': '150100', 'region': '', 'regionCode': '0', 'addr': '内蒙古呼和浩特市 电信', 'regionNames': '', 'err': ''}
223.160.223.178
{'ip': '223.160.223.178', 'pro': '北京市', 'proCode': '110000', 'city': '北京市', 'cityCode': '110000', 'region': '', 'regionCode': '0', 'addr': '北京市 广电网', 'regionNames': '', 'err': ''}
223.104.108.237
{'ip': '22

,fc,pk,ak,state,client_ip,active_time,pro,city


In [49]:
ipDataDf.to_csv("ipMap.csv", columns=["client_ip", "pro", "city"], index=False)

# 绘制可视化大屏

In [43]:
from pyecharts.charts import Line,Map,Pie, Geo, Grid, Page, Tab
from pyecharts import options as opts
from pyecharts.globals import ThemeType, ChartType, SymbolType
from pyecharts.commons.utils import JsCode

In [9]:
ipMapDf = pd.read_csv("./ipMap.csv")
dataDf = pd.merge(ipDataDf, ipMapDf, on="client_ip", how="left")
dataDf.drop_duplicates(subset=["fc", "pk", "ak", "client_ip"], inplace=True)
dataDf.head()

,fc,pk,ak,state,client_ip,active_time,pro,city
0,tn1czr,15k4eg6e,c0cbf11e3e9b,ACTIVATED,115.194.10.49,2024-02-20T02:24:57.000Z,浙江省,杭州市
1,tn1czr,15k4eg6e,c0cbf11e3e9e,ACTIVATED,112.17.238.73,2024-03-14T04:22:27.000Z,浙江省,杭州市
4,tn1czr,15k4eg6e,c0cbf11e3ec9,ACTIVATED,183.136.152.218,2024-02-23T03:04:13.000Z,浙江省,宁波市
5,tn1czr,15k4eg6e,c0cbf11e3f4b,ACTIVATED,223.104.161.212,2024-03-05T08:44:46.000Z,浙江省,宁波市
6,tn1czr,15k4eg6e,c0cbf11e3e9d,ACTIVATED,61.153.149.52,2024-01-08T03:36:51.000Z,浙江省,宁波市


In [10]:
dataDf["date"] = dataDf["active_time"].apply(lambda x : x.split("T")[0])
dataDf["date"] = pd.to_datetime(dataDf["date"], format="%Y-%m-%d")
dataDf.head()

,fc,pk,ak,state,client_ip,active_time,pro,city,date
0,tn1czr,15k4eg6e,c0cbf11e3e9b,ACTIVATED,115.194.10.49,2024-02-20T02:24:57.000Z,浙江省,杭州市,2024-02-20
1,tn1czr,15k4eg6e,c0cbf11e3e9e,ACTIVATED,112.17.238.73,2024-03-14T04:22:27.000Z,浙江省,杭州市,2024-03-14
4,tn1czr,15k4eg6e,c0cbf11e3ec9,ACTIVATED,183.136.152.218,2024-02-23T03:04:13.000Z,浙江省,宁波市,2024-02-23
5,tn1czr,15k4eg6e,c0cbf11e3f4b,ACTIVATED,223.104.161.212,2024-03-05T08:44:46.000Z,浙江省,宁波市,2024-03-05
6,tn1czr,15k4eg6e,c0cbf11e3e9d,ACTIVATED,61.153.149.52,2024-01-08T03:36:51.000Z,浙江省,宁波市,2024-01-08


In [11]:
temp = dataDf["date"].value_counts().sort_index(ascending=True)
full_date_range = pd.date_range(start="2023-12-19", end="2024-07-01")

# 重新索引 DataFrame，并填充缺失的日期数据为零
line_data = temp.reindex(full_date_range, fill_value=0)
line_data.head()

2023-12-19    2
2023-12-20    0
2023-12-21    0
2023-12-22    0
2023-12-23    0
Freq: D, Name: count, dtype: int64

In [13]:
def line_plot(x, y, yTitle, title):
    line = Line(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
    line.add_xaxis(x)
    line.add_yaxis(yTitle, y, is_smooth=True)
    # 设置全局选项
    line.set_global_opts(
        title_opts=opts.TitleOpts(title=title),
        xaxis_opts=opts.AxisOpts(
            type_="time",
            is_scale=True,
            boundary_gap=False,
            axislabel_opts=opts.LabelOpts(formatter="{yyyy}-{MM}-{dd}"),
        ),
        tooltip_opts=opts.TooltipOpts(
            trigger="item",
            formatter=JsCode(
                """
                function (params) {
                    return params.seriesName + '<br/>' + 'Date: ' + params.data[0].substring(0, 10) + '<br/>' + 'count: ' + params.data[1];
                }
                """
            ),
        ),
        datazoom_opts=[opts.DataZoomOpts()],
    )
    return line

In [14]:
proDf = dataDf["pro"].value_counts()

In [15]:
# 绘制地理分布图
def map_chart(name, count):
    c = (
        Map(init_opts=opts.InitOpts(chart_id=2, theme=ThemeType.LIGHT))
            .add("首次激活位置分布", [list(z) for z in zip(name,
                                              count)], "china")
            .set_global_opts(
            title_opts=opts.TitleOpts(title="首次激活全国的分布图",
                                      title_textstyle_opts=opts.TextStyleOpts(font_size=15,
                                                                              color='#000000')),
            visualmap_opts=opts.VisualMapOpts(max_=250),
        )
    )
    return c

In [16]:
# 激活占比饼图
def plotPie(data, title):
    pie = (
        Pie(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
        .add(title, data, radius=["20%", "65%"])
        .set_global_opts(
            title_opts=opts.TitleOpts(title=title),
            legend_opts=opts.LegendOpts(is_show=True, pos_bottom="0%", pos_left="center"),
        )
        .set_series_opts(
            label_opts=opts.LabelOpts(formatter="{b}: {d}%")
        )
    )
    return pie

In [17]:

def plotCityMap(cityDf):
    geo = (
        Geo()
        .add_schema(maptype="china")
        .add(
            "city",
            cityDf.values.tolist(),
            type_=ChartType.EFFECT_SCATTER,
            symbol_size=8,
        )
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False, formatter="{b}", font_size=8))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="全国城市分布图"),
            visualmap_opts=opts.VisualMapOpts(max_=50)
            )
    )
    return geo

In [46]:
# 将多个图表对象汇总到一个图中
cityDf = dataDf["city"].value_counts().reset_index()
cityChart = plotCityMap(cityDf)
proChart = map_chart(proDf.index.to_list(), proDf.to_list())
proPieChart = plotPie(proDf.reset_index().values.tolist(), "省份占比饼图")
pieCityData = cityDf.head(8)
pieCityData = pd.concat([pieCityData,pd.DataFrame({'city': ['其他'], 'count': [cityDf[8:]["count"].sum()]})], axis=0, ignore_index=True)
cityPie = plotPie(pieCityData.values.tolist(), "城市占比")
auxByDayLine = line_plot(line_data.index.to_list(), line_data.values.tolist(), "日首次激活数", "奥克斯日激活数")
tab = Tab()
tab.add(proChart, "奥克斯省份分布地图数据")
tab.add(proPieChart, "奥克斯省份分布占比数据")
tab.add(cityChart, "奥克斯城市分布地图数据")
tab.add(cityPie, "奥克斯城市分布占比数据")
tab.add(auxByDayLine, "奥克斯每日首次激活量")
tab.render("aux_day_visualMap.html")


'/Users/liangzonglin/workplace/project/baidu/bce-iot/du-home-log/dags/temp/task42 aux_analysis/aux_day_visualMap.html'